# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы.
Нужно построить модель для задачи классификации, которая выберет подходящий тариф.

Постройте модель с максимально большим значением *accuracy*.
Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75.

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.dummy import DummyClassifier

In [2]:
df = pd.read_csv('users_behavior.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
df

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


Вывод: Данные без пропусков, предобработка не требуется:

## Разбейте данные на выборки

In [5]:
features = df.drop(['is_ultra'], axis=1) # признаки 
target = df['is_ultra'] # целевой признак

### Делим на тренировочную и тестовую 60\40 

In [6]:
(features_train, features_test, target_train, target_test) = (train_test_split(features, target, test_size=0.4, 
                                                                               random_state=12345))

### Делим на валидационную и тестовую 50\50

In [7]:
(features_valid, features_test, target_valid, target_test) = (train_test_split(features_test,target_test, test_size = 0.5,
                                                                               random_state = 12345))

## Исследуйте модели

### Решающее дерево

In [8]:
best_model = None
best_result = 0
best_depth = 0
for depth in range(1, 50):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    result = model.score(features_valid, target_valid)
    if result > best_result:
        best_model_1 = model
        best_result = result
        best_depth = depth

print(f'Accuaracu наилучшей модели на валидационной выборке: {best_result:.3f}')
print(f'Глубина дерева: {best_depth}')

Accuaracu наилучшей модели на валидационной выборке: 0.785
Глубина дерева: 3


### Случайный лес

In [9]:
best_model = None
best_result = 0
best_est = 0
best_depth = 0
for est in range(1,50):
    for depth in range (1, 11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train,target_train)
        result = model.score(features_valid, target_valid)
        if result > best_result:
            best_model_2 = model
            best_result = result
            best_est = est
            best_depth = depth

print(f'Accuaracu наилучшей модели на валидационной выборке: {best_result:.3f}')
print(f'Количество деревьев: {best_est}')
print(f'Максимальная глубина: {depth}')

Accuaracu наилучшей модели на валидационной выборке: 0.809
Количество деревьев: 40
Максимальная глубина: 10


#### Функция Gridsearchcv

In [10]:
clf = RandomForestClassifier()
parametrs = { 'n_estimators': range (10, 51, 10),
              'max_depth': range (1,13, 2),
              'min_samples_leaf': range (1,8),
              'min_samples_split': range (2,10,2) }
grid = GridSearchCV(clf, parametrs, cv=5)
grid.fit(features_train, target_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': range(1, 13, 2),
                         'min_samples_leaf': range(1, 8),
                         'min_samples_split': range(2, 10, 2),
                         'n_estimators': range(10, 51, 10)})

In [11]:
grid.best_params_

{'max_depth': 11,
 'min_samples_leaf': 3,
 'min_samples_split': 6,
 'n_estimators': 10}

In [12]:
clf1 = RandomForestClassifier(max_depth=9, n_estimators=40, min_samples_leaf=2, min_samples_split=6)
clf1.fit(features_train,target_train)
clf1.score(features_valid, target_valid).round(3)

0.801

### Логистическая регрессия 

In [13]:
model = LogisticRegression()
model.fit(features_train, target_train)
result = model.score(features_valid, target_valid)
print(f'Accuaracu модели логистической регрессии на валидационной выборке: {result:.3f}')

Accuaracu модели логистической регрессии на валидационной выборке: 0.711


### Лучшая модель 

In [14]:
print(f'Accuaracu лучшей модели: {best_result:.3f}')
print(f'Лучшая модель: {best_model_2}')

Accuaracu лучшей модели: 0.809
Лучшая модель: RandomForestClassifier(max_depth=8, n_estimators=40, random_state=12345)


## Проверьте модель на тестовой выборке

In [15]:
prediction_test = best_model_2.predict(features_test)

In [16]:
score_best_model_test = accuracy_score(prediction_test, target_test)

In [17]:
print(f'Accuaracu модели на тестовой выборке: {score_best_model_test:.3f}')

Accuaracu модели на тестовой выборке: 0.796


### Вывод:

`RandomForestClassifier(max_depth=8, n_estimators=40, random_state=12345)` на валидационной и тестовой выборке, показывает одинаково хороший результат

## Проверьте модели на адекватность

In [18]:
print(df['is_ultra'].value_counts() / df.shape[0])

dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(features_train, target_train)
DummyClassifier(strategy='most_frequent')
dummy_clf.predict(features_test)
print(dummy_clf.score(prediction_test, target_test))

0    0.693528
1    0.306472
Name: is_ultra, dtype: float64
0.6842923794712286


## Вывод по проекту

Исследуя модели классификации `DecisionTreeClassifier` `RandomForestClassifier` `LogisticRegression`. Выбрали модель с лучшим Accuaracu:0.8087, RandomForestClassifier с гиперпараметрами max_depth=8, n_estimators=40.

Проверив модель на тестовой выборке получили Accuaracu (0.796) близкое к значению Accuaracu полученному на валидационный выборке, но не превышающем его, из чего следует что модель не подверглась переобучению и недообучению